# DFS5.2
The time units attribute of DFS5.2 is missing.  
This recipe shows how to concatenate multiple years.

In [ ]:
# Import modules
import intake
import xarray as xr

# Import DFS5_2 entry
cat = intake.open_catalog("../catalog.yaml")
DFS = cat.NOCS["DFS5.2"]


# Function to add time units
def add_time_units(ds):
    """
    Find out year from Identification attribute,
    the add units to the time coordinate.
    """
    filename = ds.attrs["Identification"].split(": ")[-1]
    year = filename.split("_y")[-1].replace(".nc", "")
    ds.time.attrs["units"] = f"days since {year}"
    return xr.decode_cf(ds)


# Xarray arguments
xarray_kwargs = dict(
    **DFS.describe()["args"][
        "xarray_kwargs"
    ],  # Use the xarray arguments set in the catalog
    preprocess=add_time_units,  # Add preprocess argument
)

# Loop over years
years = range(1990, 2000)
datasets = []
print(f"Reading {DFS.description}", end=": ")
for year in years:
    print(year, end=", " if year != years[-1] else ".\n")
    datasets += [DFS(year=year, xarray_kwargs=xarray_kwargs).to_dask()]

# Concatenate
# See http://xarray.pydata.org/en/stable/io.html#reading-multi-file-datasets
concat_kwargs = dict(data_vars="minimal", coords="minimal", compat="override")
ds = xr.concat(datasets, "time", **concat_kwargs)
ds